In [31]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
import random
import struct
import torch
import torchvision
from torchvision import transforms,datasets
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import PIL
from PIL import Image
import shutil
from torch.utils.data.sampler import SubsetRandomSampler
import torch.backends.cudnn as cudnn
import torchvision.models as models
from tqdm import tqdm

In [32]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [33]:
torch.backends.cudnn.deterministic = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
path_dataset_new = '/content/gdrive/My Drive/dataset_new_synergy'

In [34]:
image_datasets =datasets.ImageFolder(path_dataset_new,transform_train)

batch_size = 15
validation_split = .1
shuffle_dataset = True
random_seed= 20
validation_batch_size = 1

# Creating data indices for training and validation splits:
dataset_size = len(image_datasets)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, 
                                                    sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(image_datasets, batch_size=validation_batch_size,
                                                     sampler=valid_sampler)

In [35]:
model = torchvision.models.resnet18(pretrained=True, progress=True)
model.fc = nn.Linear(in_features=512, out_features=2, bias=True)

In [36]:
model.to(device)
model.double()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes) 
    return y[labels]

In [40]:
torch.cuda.empty_cache()
epochs = 8
accuracy  = []
for epoch in tqdm(range(epochs)):
    model.train()
    for data in train_loader:
        inputs,labels=data
        inputs = inputs.to(device)
        inputs=inputs.double()
        inputs=Variable(inputs)
        labels = labels.to(device)
        labels=Variable(labels)
        outputs=model.forward(inputs)
        loss=criterion(outputs,labels)
        model.zero_grad()
        loss.backward()
        optimizer.step()
#==============================================================================================================================#
    acc = 0
    Total_images = 0
    model.eval()
    for data in validation_loader:
        inputs,labels=data
        Total_images += len(inputs)
        inputs = inputs.to(device)
        inputs=inputs.double()
        inputs=Variable(inputs)
        labels = labels.to(device)
        labels=Variable(labels)
        outputs=model.forward(inputs)
        if outputs.argmax().item() == labels.item():
            acc+=1
    Accuracy = acc/Total_images
    accuracy.append(Accuracy)
    print("Accuracy on validation set after {} epoch is {}".format(epoch,Accuracy))
    l = len(accuracy)
    if len(accuracy) == 1:
        pass
    if accuracy[l-1] > accuracy[l-2]:
      model_save_name = 'resnet_Trained_on_synergy_dataset.pt'
      path = F"/content/gdrive/My Drive/{model_save_name}" 
      torch.save(model.state_dict(), path)  
      print("model parameters saved!!")





  0%|          | 0/8 [00:00<?, ?it/s]



 12%|█▎        | 1/8 [00:18<02:10, 18.58s/it]

Accuracy on validation set after 0 epoch is 0.9166666666666666






 25%|██▌       | 2/8 [00:37<01:51, 18.57s/it]

Accuracy on validation set after 1 epoch is 0.8928571428571429
Accuracy on validation set after 2 epoch is 0.9404761904761905






 38%|███▊      | 3/8 [00:56<01:33, 18.67s/it]

...

model parameters saved!!






 88%|████████▊ | 7/8 [02:10<00:18, 18.70s/it]

Accuracy on validation set after 6 epoch is 0.9166666666666666






100%|██████████| 8/8 [02:29<00:00, 18.69s/it]

Accuracy on validation set after 7 epoch is 0.8333333333333334


In [41]:
model_test = torchvision.models.resnet18(pretrained=False, progress=True)
model_test.fc = nn.Linear(in_features=512, out_features=2, bias=True)
model_test.load_state_dict(torch.load('/content/gdrive/My Drive/resnet_Trained_on_synergy_dataset.pt'))
model_test.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [42]:
test_dataset_new = '/content/gdrive/My Drive/test_data_new'

In [43]:
test_datasets =datasets.ImageFolder(test_dataset_new,transform_test)

test_loader = torch.utils.data.DataLoader(test_datasets, batch_size = 1,
                                                     shuffle = True)

In [44]:
    acc_test = 0
    Total_images_test = 0
    model_test.eval()
    model_test.to(device)
    model_test.double()
    for data in validation_loader:
        inputs,labels=data
        Total_images_test += len(inputs)
        inputs = inputs.to(device)
        inputs=inputs.double()
        inputs=Variable(inputs)
        labels = labels.to(device)
        labels=Variable(labels)
        outputs=model_test.forward(inputs)
        if outputs.argmax().item() == labels.item():
            acc_test+=1

In [45]:
Accuracy_test = (acc_test/Total_images_test)*100

In [46]:
Accuracy_test

91.66666666666666